In [1]:
from tool import *

In [2]:
with open('D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)

In [3]:
thisweek = get_weekname(datetime.date.today())

In [4]:
sr['Week Name'] = sr['Ship Date'].map(get_weekname)

In [5]:
sr = sr.reset_index().merge(site[['To Site', 'Region', 'Country', 'Subsidiary']])

In [6]:
sr.loc[sr['Division']=='PC', 'SCM'] = sr.loc[sr['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sr.loc[sr['Division']=='PC', 'MKT'] = sr.loc[sr['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sr.loc[sr['Division']=='MNT', 'SCM'] = sr.loc[sr['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sr.loc[sr['Division']=='MNT', 'MKT'] = sr.loc[sr['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].min())
us_b2b = sr.loc[sr['To Site'].str.contains('ENUS') & (sr['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sr.loc[sr['To Site'].str.contains('ENUS') & (sr['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2b

In [7]:
sr = sr[sr['Week Name']==get_weekname_from(thisweek, -1)].fillna('N/A').pivot_table('Ship', index=['From Site', 'BL No', 'Region', 'Country', 'Subsidiary', 'To Site', 'Mapping Model.Suffix', 'PO No.', 'PO1 No', 'SO No.', 'Final Destination', 'Ship Date', 'SCM', 'MKT'], aggfunc=sum).reset_index()
# sr = sr[sr['Week Name'].isin([get_weekname_from(thisweek, 0)])].fillna('N/A').pivot_table('Ship', index=['From Site', 'BL No', 'Region', 'Country', 'Subsidiary', 'To Site', 'Mapping Model.Suffix', 'PO No.', 'PO1 No', 'SO No.', 'Final Destination', 'Ship Date', 'SCM', 'MKT'], aggfunc=sum).reset_index()

In [8]:
sr = sr[sr['From Site'].str.contains('TCL')]

# 매출 담당자
### 1) EU : 전유미 선임
### 2) MEA : 오설 책임
### 3) SEA : 현은지 선임
### 4) NA : 김정미 사원
### 5) SCA : 최경남 책임
### 6) Korea : 이창훈 책임, 김윤정 선임, 김준현 사원, 조희진 선임, 송예슬 사원, 김승현 기정, 양서하 사원, 김운 책임, 정경훈 조장, 이태균 선임, 김종철 책임

#### 이창훈 책임 changhoon123.lee.lee@lgepartner.com
#### 조희진 선임 hyijin.cho@lgepartner.com
#### 송예슬 사원 yeseul.song@lgepartner.com

In [9]:
sr

,From Site,BL No,Region,Country,Subsidiary,To Site,Mapping Model.Suffix,PO No.,PO1 No,SO No.,Final Destination,Ship Date,SCM,MKT,Ship
78,OS_TCL_CN_P,78406483621,SEA,Singapore,LGESL,EASL_DO,24MR400-B.AHKQ,JGMZ24003152,1033429,380175279,BDDAC,2024-03-08,김송이,임채구,480.0
79,OS_TCL_CN_P,78406483621,SEA,Singapore,LGESL,EASL_DO,27MR400-B.AHKQ,JGMZ24003153,1033429,380175279,BDDAC,2024-03-08,김송이,임채구,127.0
80,OS_TCL_CN_P,PLISZ4F02721,EU,Hungary,LGEMK,EEMK,27MR400-B.AEUQ,JGMZ24003372,1076040,380175405,HUSZ2,2024-03-04,송명규,장원영,512.0
81,OS_TCL_CN_P,PLISZ4F02731,SEA,Philippines,LGEPH,EACM,22MR410-B.APHQ,JGMZ24003344,1021311,380175367,PHMNL,2024-03-05,김송이,박진아,588.0
82,OS_TCL_CN_P,PLISZ4F02731,SEA,Philippines,LGEPH,EACM,24MR400-B.APHQ,JGMZ24003345,1021311,380175367,PHMNL,2024-03-05,김송이,박진아,3360.0
83,OS_TCL_CN_P,PLISZ4F02769,MEA,Turkiye,LGETK,EMTK,24MR400-B.APDQ,JGMZ24003460,1399752,380175461,TRTUZ,2024-03-04,오석현,최성용,800.0
84,OS_TCL_CN_P,PLISZ4F02769,MEA,Turkiye,LGETK,EMTK,27MR400-B.APDQ,JGMZ24004524,1402095,380175811,TRTUZ,2024-03-04,오석현,최성용,512.0
85,OS_TCL_CN_P,PLISZ4F02858,MEA,UAE,LGEGF,EMGF,22MR410-B.AMAQ,JGMZ24004349,1195935,380175713,AEJEA,2024-03-06,오석현,박세리,1680.0
86,OS_TCL_CN_P,PLISZ4F02858,MEA,UAE,LGEGF,EMGF,24MR400-B.AMAQ,JGMZ24004350,1195935,380175713,AEJEA,2024-03-06,오석현,박세리,800.0


In [10]:
bl_num = ['78406483621']
c1 = sr['BL No'].isin(bl_num)

In [11]:
sr[c1].to_clipboard(index=False)
sr[c1]

,From Site,BL No,Region,Country,Subsidiary,To Site,Mapping Model.Suffix,PO No.,PO1 No,SO No.,Final Destination,Ship Date,SCM,MKT,Ship
78,OS_TCL_CN_P,78406483621,SEA,Singapore,LGESL,EASL_DO,24MR400-B.AHKQ,JGMZ24003152,1033429,380175279,BDDAC,2024-03-08,김송이,임채구,480.0
79,OS_TCL_CN_P,78406483621,SEA,Singapore,LGESL,EASL_DO,27MR400-B.AHKQ,JGMZ24003153,1033429,380175279,BDDAC,2024-03-08,김송이,임채구,127.0


In [12]:
# bl_num = sr.loc[i]['BL No']
region = sr[c1]['Region'].unique()[0]
country = sr[c1]['Country'].unique()[0]
weekname = sr[c1]['Ship Date'].map(get_weekname).unique()[0]

email_subject = f'중국 ODM社 TCL 모니터 제품 중계무역 {region}향({country}) 매출화 요청 {bl_num}'
email_body = f'''안녕하십니까?

아래의 건으로 {weekname}주차 {region} 지역인 {country}향 중계무역 매출화 요청 드립니다.
상세한 내용은 첨부의 B/L 참조하시기 바랍니다.
'''
email_subject_kor = f'제목 : 중국 ODM社 TCL 모니터 제품 한국향 통관 요청 {bl_num}'
email_body_kor = f'''
안녕하십니까?

수신 : Pantos 이창훈 책임님
아래의 건으로 중국 모니터 업체인 TCL사로부터 {weekname}주차 한국으로 선적된 모니터 제품 Port에 도착되면 통관 진행 요청 드립니다.
상세한 내용은 첨부의 선적 서류 참조하시기 바랍니다.

양서하 사원님

CC request 부탁 드립니다.

정경훈 조장님

입고 검사 진행해주시기 바랍니다.

'''    

In [13]:
if sr[c1]['Country'].values[0] == 'Korea':
    print(email_subject_kor)
    print(email_body_kor)
elif region in ['SEA', 'INDIA']:
    print(email_subject)
    print(email_body)
    nw = pd.read_excel(get_filename(), skiprows=1)['Net Wt'].dropna().values[-1]
    print(f'Net Weight: {nw} KG')
elif sr[c1]['Country'].values[0] == 'Greece':
    rint(email_subject)
    print(email_body)
    nw = pd.read_excel(get_filename(), skiprows=1)['Net Wt'].dropna().values[-1]
    print(f'Net Weight: {nw} KG')
else:
    print(email_subject)
    print(email_body)

중국 ODM社 TCL 모니터 제품 중계무역 SEA향(Singapore) 매출화 요청 ['78406483621']
안녕하십니까?

아래의 건으로 2024-03-04(W10)주차 SEA 지역인 Singapore향 중계무역 매출화 요청 드립니다.
상세한 내용은 첨부의 B/L 참조하시기 바랍니다.

Net Weight: 2051.2 KG


## 인도법인 매출 시 수신자 추가

kushik.kaushal@lge.com (LGEIL IT DP)
jw20.kim@lge.com (LGEIL IT PM)
sanjeev.mittal@lge.com (LGEIL PH)
bryan1223.kim@lge.com (신규 본사담당자)

Dear EAIL team

Please see the attached shipping documents and S/N list for your reference.